In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

# Import and Build Model

In [2]:
from deeprte.experiment import Trainer
import jax
import haiku as hk

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
source_dir = "/workspaces/deeprte/rte_data/rte_data/matlab/train-delta"
data_name_list = ["train_delta_1.mat","train_delta_2.mat"]

In [4]:
from deeprte.config import get_config
config = get_config()
config = config.experiment_kwargs.config
config.dataset.source_dir = source_dir
config.dataset.data_name_list = data_name_list
rng = hk.PRNGSequence(jax.random.PRNGKey(42))
e = Trainer(mode = "train", init_rng=0, config = config)

# Data Shape Visualization

In [5]:
import tensorflow as tf
tf.nest.map_structure(lambda x: x.shape, (e.tf_data))

{'sigma': TensorShape([600, 1600, 2]),
 'psi_label': TensorShape([600, 38400]),
 'scattering_kernel': TensorShape([600, 38400, 24]),
 'self_scattering_kernel': TensorShape([600, 24, 24]),
 'boundary': TensorShape([600, 1920]),
 'position_coords': TensorShape([1600, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([38400, 4]),
 'boundary_coords': TensorShape([1920, 4]),
 'boundary_weights': TensorShape([1920]),
 'velocity_weights': TensorShape([24])}

## Train Input

In [6]:
train_input = e._build_train_input()
tf.nest.map_structure(lambda x: x.shape, next(train_input))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'sigma': (1, 8, 1600, 2),
 'psi_label': (1, 8, 500),
 'scattering_kernel': (1, 8, 500, 24),
 'self_scattering_kernel': (1, 8, 24, 24),
 'boundary': (1, 8, 1920),
 'position_coords': (1, 1600, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 500, 4),
 'boundary_coords': (1, 1920, 4),
 'boundary_weights': (1, 1920),
 'velocity_weights': (1, 24)}

## Val Input

In [7]:
val_input = e._build_eval_input()
tf.nest.map_structure(lambda x: x.shape, next(val_input))

{'sigma': (1, 8, 1600, 2),
 'psi_label': (1, 8, 38400),
 'scattering_kernel': (1, 8, 38400, 24),
 'self_scattering_kernel': (1, 8, 24, 24),
 'boundary': (1, 8, 1920),
 'position_coords': (1, 1600, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 38400, 4),
 'boundary_coords': (1, 1920, 4),
 'boundary_weights': (1, 1920),
 'velocity_weights': (1, 24)}

## Dummy Input for Model Init

In [8]:
ds = e._build_dummy_input()

In [9]:
tf.nest.map_structure(lambda x: x.shape, ds)

{'sigma': (1, 1, 1600, 2),
 'psi_label': (1, 1, 1),
 'scattering_kernel': (1, 1, 1, 24),
 'self_scattering_kernel': (1, 1, 24, 24),
 'boundary': (1, 1, 1920),
 'position_coords': (1, 1600, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 1, 4),
 'boundary_coords': (1, 1920, 4),
 'boundary_weights': (1, 1920),
 'velocity_weights': (1, 24)}

# Test Train Step

In [10]:
from jaxline import utils as jl_utils
init_rng = jl_utils.bcast_local_devices(next(rng))
e._initialize_training()
iterator=tf.data.Dataset.from_tensors(ds).as_numpy_iterator()
e._train_input = iterator

In [11]:
scalars = e.step(jl_utils.bcast_local_devices(0), jl_utils.bcast_local_devices(next(rng)))

In [15]:
scalars

{'learning_rate': DeviceArray(0.0008, dtype=float32),
 'train_mse': DeviceArray(0.06758966, dtype=float32),
 'train_rmspe': DeviceArray(64.88431, dtype=float32)}

# Test Evaluation

In [12]:
e._eval_input = iterator

In [16]:
metrics = e.evaluate(jl_utils.bcast_local_devices(0), jl_utils.bcast_local_devices(next(rng)))

In [17]:
metrics

{'eval_mse': 0.0852733626961708, 'eval_rmspe': 124.36205291748047}